In [1]:
import tkinter as tk
from tkinter import filedialog
import numpy as np
import matplotlib
matplotlib.use("TkAgg")

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from stl import mesh


# -----------------------------
# Rotation helpers
# -----------------------------
def rotate_x(points, angle):
    rad = np.deg2rad(angle)
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(rad), -np.sin(rad)],
        [0, np.sin(rad),  np.cos(rad)]
    ])
    return points @ Rx.T


def rotate_y(points, angle):
    rad = np.deg2rad(angle)
    Ry = np.array([
        [ np.cos(rad), 0, np.sin(rad)],
        [ 0,           1, 0          ],
        [-np.sin(rad), 0, np.cos(rad)]
    ])
    return points @ Ry.T


def rotate_z(points, angle):
    rad = np.deg2rad(angle)
    Rz = np.array([
        [np.cos(rad), -np.sin(rad), 0],
        [np.sin(rad),  np.cos(rad), 0],
        [0,            0,           1]
    ])
    return points @ Rz.T


# -----------------------------
# Main Application
# -----------------------------
class STLViewerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Tkinter STL Viewer – Smooth Rotation")

        self.mesh_data = None
        self.original_vectors = None
        self.center = None
        self.limits = None

        # Matplotlib Figure
        self.fig = Figure(figsize=(6, 6))
        self.ax = self.fig.add_subplot(111, projection="3d")

        self.canvas = FigureCanvasTkAgg(self.fig, master=root)
        self.canvas.get_tk_widget().grid(row=0, column=0, rowspan=6, padx=5, pady=5)

        # Rotation variables (float!)
        self.rx = tk.DoubleVar(value=0.0)
        self.ry = tk.DoubleVar(value=0.0)
        self.rz = tk.DoubleVar(value=0.0)

        # Controls
        tk.Button(root, text="Load STL", command=self.load_stl)\
            .grid(row=0, column=1, sticky="ew", padx=5)

        self.rot_x = tk.Scale(
            root, from_=-180, to=180,
            orient=tk.HORIZONTAL,
            resolution=0.2,     # 🔑 smooth step
            variable=self.rx,
            label="Rotate X (°)",
            command=self.update_plot
        )

        self.rot_y = tk.Scale(
            root, from_=-180, to=180,
            orient=tk.HORIZONTAL,
            resolution=0.2,
            variable=self.ry,
            label="Rotate Y (°)",
            command=self.update_plot
        )

        self.rot_z = tk.Scale(
            root, from_=-180, to=180,
            orient=tk.HORIZONTAL,
            resolution=0.2,
            variable=self.rz,
            label="Rotate Z (°)",
            command=self.update_plot
        )

        self.rot_x.grid(row=1, column=1, sticky="ew", padx=5)
        self.rot_y.grid(row=2, column=1, sticky="ew", padx=5)
        self.rot_z.grid(row=3, column=1, sticky="ew", padx=5)

    # -----------------------------
    # Load STL File
    # -----------------------------
    def load_stl(self):
        file_path = filedialog.askopenfilename(
            filetypes=[("STL files", "*.stl")]
        )
        if not file_path:
            return

        self.mesh_data = mesh.Mesh.from_file(file_path)
        self.original_vectors = np.copy(self.mesh_data.vectors)

        all_points = self.original_vectors.reshape(-1, 3)

        # True object center
        self.center = all_points.mean(axis=0)

        # Fixed axis limits
        min_vals = all_points.min(axis=0)
        max_vals = all_points.max(axis=0)

        size = max(max_vals - min_vals) * 1.5

        self.limits = {
            "x": (self.center[0] - size / 2, self.center[0] + size / 2),
            "y": (self.center[1] - size / 2, self.center[1] + size / 2),
            "z": (self.center[2] - size / 2, self.center[2] + size / 2),
        }

        # Reset sliders
        self.rx.set(0)
        self.ry.set(0)
        self.rz.set(0)

        self.update_plot()

    # -----------------------------
    # Update Plot
    # -----------------------------
    def update_plot(self, event=None):
        if self.mesh_data is None:
            return

        # Read smooth float values
        rx = self.rx.get()
        ry = self.ry.get()
        rz = self.rz.get()

        points = self.original_vectors.reshape(-1, 3)

        # Rotate around object center
        points = points - self.center
        points = rotate_x(points, rx)
        points = rotate_y(points, ry)
        points = rotate_z(points, rz)
        points = points + self.center

        vectors = points.reshape((-1, 3, 3))

        self.ax.clear()

        collection = Poly3DCollection(vectors, alpha=0.7)
        collection.set_facecolor("lightsteelblue")
        self.ax.add_collection3d(collection)

        self.ax.set_xlim(self.limits["x"])
        self.ax.set_ylim(self.limits["y"])
        self.ax.set_zlim(self.limits["z"])
        self.ax.set_box_aspect([1, 1, 1])

        self.ax.set_xlabel("X")
        self.ax.set_ylabel("Y")
        self.ax.set_zlabel("Z")

        self.canvas.draw_idle()  #  smoother redraw


# -----------------------------
# Run Application
# -----------------------------
if __name__ == "__main__":
    root = tk.Tk()
    app = STLViewerApp(root)
    root.mainloop()

In [ ]:
import tkinter as tk
from tkinter import filedialog
import numpy as np
import matplotlib
matplotlib.use("TkAgg")

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from stl import mesh


# -----------------------------
# Rotation helpers
# -----------------------------
def rotate_x(points, angle):
    rad = np.deg2rad(angle)
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(rad), -np.sin(rad)],
        [0, np.sin(rad),  np.cos(rad)]
    ])
    return points @ Rx.T


def rotate_y(points, angle):
    rad = np.deg2rad(angle)
    Ry = np.array([
        [ np.cos(rad), 0, np.sin(rad)],
        [ 0,           1, 0          ],
        [-np.sin(rad), 0, np.cos(rad)]
    ])
    return points @ Ry.T


def rotate_z(points, angle):
    rad = np.deg2rad(angle)
    Rz = np.array([
        [np.cos(rad), -np.sin(rad), 0],
        [np.sin(rad),  np.cos(rad), 0],
        [0,            0,           1]
    ])
    return points @ Rz.T


# -----------------------------
# Main Application
# -----------------------------
class STLViewerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Tkinter STL Viewer – Rotate & Translate")

        self.mesh_data = None
        self.original_vectors = None
        self.center = None
        self.limits = None
        self.size = None

        # Matplotlib Figure
        self.fig = Figure(figsize=(6, 6))
        self.ax = self.fig.add_subplot(111, projection="3d")

        self.canvas = FigureCanvasTkAgg(self.fig, master=root)
        self.canvas.get_tk_widget().grid(row=0, column=0, rowspan=9, padx=5, pady=5)

        # Rotation variables
        self.rx = tk.DoubleVar(value=0.0)
        self.ry = tk.DoubleVar(value=0.0)
        self.rz = tk.DoubleVar(value=0.0)

        # Translation variables
        self.tx = tk.DoubleVar(value=0.0)
        self.ty = tk.DoubleVar(value=0.0)
        self.tz = tk.DoubleVar(value=0.0)

        # Controls
        tk.Button(root, text="Load STL", command=self.load_stl)\
            .grid(row=0, column=1, sticky="ew", padx=5)

        # Rotation sliders
        self._make_slider("Rotate X (°)", self.rx, 1)
        self._make_slider("Rotate Y (°)", self.ry, 2)
        self._make_slider("Rotate Z (°)", self.rz, 3)

        # Translation sliders (range set after STL load)
        self.s_tx = self._make_slider("Translate X", self.tx, 4)
        self.s_ty = self._make_slider("Translate Y", self.ty, 5)
        self.s_tz = self._make_slider("Translate Z", self.tz, 6)

    def _make_slider(self, label, var, row):
        s = tk.Scale(
            self.root,
            from_=-180, to=180,
            orient=tk.HORIZONTAL,
            resolution=0.2,
            variable=var,
            label=label,
            command=self.update_plot
        )
        s.grid(row=row, column=1, sticky="ew", padx=5)
        return s

    # -----------------------------
    # Load STL
    # -----------------------------
    def load_stl(self):
        file_path = filedialog.askopenfilename(
            filetypes=[("STL files", "*.stl")]
        )
        if not file_path:
            return

        self.mesh_data = mesh.Mesh.from_file(file_path)
        self.original_vectors = np.copy(self.mesh_data.vectors)

        all_points = self.original_vectors.reshape(-1, 3)

        self.center = all_points.mean(axis=0)

        min_vals = all_points.min(axis=0)
        max_vals = all_points.max(axis=0)

        self.size = max(max_vals - min_vals)

        # Fixed axis limits
        pad = self.size * 0.75
        self.limits = {
            "x": (self.center[0] - self.size - pad, self.center[0] + self.size + pad),
            "y": (self.center[1] - self.size - pad, self.center[1] + self.size + pad),
            "z": (self.center[2] - self.size - pad, self.center[2] + self.size + pad),
        }

        # Translation slider ranges (soft, proportional)
        for s in (self.s_tx, self.s_ty, self.s_tz):
            s.config(from_=-self.size, to=self.size)

        # Reset all controls
        self.rx.set(0)
        self.ry.set(0)
        self.rz.set(0)
        self.tx.set(0)
        self.ty.set(0)
        self.tz.set(0)

        self.update_plot()

    # -----------------------------
    # Update Plot
    # -----------------------------
    def update_plot(self, event=None):
        if self.mesh_data is None:
            return

        rx, ry, rz = self.rx.get(), self.ry.get(), self.rz.get()
        tx, ty, tz = self.tx.get(), self.ty.get(), self.tz.get()

        points = self.original_vectors.reshape(-1, 3)

        # Rotate around center
        points = points - self.center
        points = rotate_x(points, rx)
        points = rotate_y(points, ry)
        points = rotate_z(points, rz)
        points = points + self.center

        # Apply translation
        points = points + np.array([tx, ty, tz])

        vectors = points.reshape((-1, 3, 3))

        self.ax.clear()

        mesh_poly = Poly3DCollection(vectors, alpha=0.7)
        mesh_poly.set_facecolor("lightsteelblue")
        self.ax.add_collection3d(mesh_poly)

        self.ax.set_xlim(self.limits["x"])
        self.ax.set_ylim(self.limits["y"])
        self.ax.set_zlim(self.limits["z"])
        self.ax.set_box_aspect([1, 1, 1])

        self.ax.set_xlabel("X")
        self.ax.set_ylabel("Y")
        self.ax.set_zlabel("Z")

        self.canvas.draw_idle()


# -----------------------------
# Run Application
# -----------------------------
if __name__ == "__main__":
    root = tk.Tk()
    app = STLViewerApp(root)
    root.mainloop()